In [6]:
import pandas as pd
import csv
import numpy as np
import re
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances
import fastcluster
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import fcluster
import gensim 
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import sys
from nltk.stem.porter import PorterStemmer
import math
import hdbscan
import warnings
warnings.filterwarnings('ignore')
sys.setrecursionlimit(100000)

# Cluster

In [7]:
#get all possible iterations of the username
def get_codes_from_username(username):
    usernames = []
    usernames.append(re.escape(username))
    split_regex = re.compile(" ")
    tokens = re.split(split_regex, username)
    if len(tokens) > 1:
        count = 0
        initials = ""
        total_string = ""
        for token in tokens:
            usernames.append(re.escape(token))
            total_string += token
            if count != 0:
                usernames.append(re.escape(total_string))
            count += 1
            if len(token) > 0:
                initials += token[0]
        usernames.append(re.escape(initials))
    
    last_index_used = 0
    for i in range(len(username)-1):
        c = ""
        nc = ""
        try:
            c = unicode(username[i], 'utf-8')
            nc = unicode(username[i+1], 'utf-8')
        except:
            continue
        if c.islower() and (nc.isupper() or nc.isnumeric()):
            usernames.append(re.escape(username[last_index_used: i+1]))
            usernames.append(re.escape(username[i+1:]))
            last_index_used = i+1
        elif c.isupper() and nc.isnumeric():
            usernames.append(re.escape(username[last_index_used: i+1]))
            usernames.append(re.escape(username[i+1:]))
            last_index_used = i+1
        elif c.isnumeric() and  not nc.isnumeric():
            usernames.append(re.escape(username[last_index_used: i+1]))
            usernames.append(re.escape(username[i+1:]))
            last_index_used = i+1
    return usernames

def get_search_regex(possible_code_bases):
    if possible_code_bases == None:
        return "(" + username + ").*[0-9]*"
    search_str = "("
    for base in possible_code_bases:
        search_str += base + "|"
    search_str = search_str[0:-1] + ").*[0-9]*"
    return search_str

In [8]:
videos = []
count = 0
with open("sentences_en1.tsv", 'rt') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        count += 1
        regexp_str = get_search_regex(get_codes_from_username(row[2]))
        regexp = re.compile(regexp_str)
        if re.search(regexp, row[3]):
            videos.append([row[0], row[1], row[2], row[3]])
        
videos_en_new = pd.DataFrame(videos, columns=['id', 'description', 'channelTitle', 'sentence'])
del videos

In [9]:
print("Total number of sentences: " + str(count))
print("Number of videos with a possible coupon code: " + str(videos_en_new.shape[0]))
print("Percentage: " + str(videos_en_new.shape[0]/(count*1.0)))

Total number of sentences: 1138630
Number of videos with a possible coupon code: 170473
Percentage: 0.149717643133


In [19]:
with open('temp.tsv', 'wt') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
    count = 0
    for index, row in videos_en_new.iterrows():
        shouldPrint = False
        iden = row["id"]
        if iden:
            iden = iden.encode('utf-8')
        try:
            desc = row["description"].encode('utf-8')
        except:
            desc = row['description']
        try:
            chanTitle = row["channelTitle"].encode('utf-8')
        except:
            chanTitle = row["channelTitle"]
            shouldPrint = True
        try:
            sent = row['sentence'].encode('utf-8')
        except:
            desc = row['sentence']
            
        if shouldPrint:
            writer.writerow([iden, desc, chanTitle, sent])
            break
    print(count)

0
